<a href="https://colab.research.google.com/github/clajps0n1/Deep-RL-course/blob/main/notebooks/unit1/practice1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Baseline, swig, gymnasium and huggingface

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
!pip install swig

In [ ]:
!pip install gymnasium[box2d]

In [ ]:
!pip install huggingface_sb3

Dependencies for generating the video

In [ ]:
!sudo apt-get update
!apt install python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Import packages

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

Create the environment

In [ ]:
env = gymnasium.make("CartPole-v1")
env.reset()

print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation


print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action

Vectorized envirnment

In [ ]:
env  = make_vec_env('CartPole-v1', n_envs=8)
env.reset()

Instantiate the agent

In [ ]:
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1
)

Train the agent

In [ ]:
model.learn(total_timesteps=200000)

Save model

In [ ]:
model_name = "ppo-CartPole-v1"
model.save(model_name)

Evaluate the agent

In [ ]:
eval_env = Monitor(gymnasium.make("CartPole-v1", render_mode='rgb_array'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

Publish the model on the hub

In [ ]:
notebook_login()

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder

def _vec_video_recorder__getattr(self, name):
    # Fix a bug in push_to_hub.py where it looks for video_recorder.path
    if "video_recorder" == name:
        return self
    if "path" == name:
        return self.video_path
    if None is _orig_vec_video_recorder_getattr:
        raise AttributeError(name)
    return _orig_vec_video_recorder_getattr(self, name)


_orig_vec_video_recorder_getattr = getattr(VecVideoRecorder, "__getattr__", None)
setattr(VecVideoRecorder, "__getattr__", _vec_video_recorder__getattr)

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from huggingface_sb3 import package_to_hub

# Define the name of the environment
env_id = "CartPole-v1"

# TODO: Define the model architecture we used
model_architecture = "PPO"

## Define a repo_id
repo_id = "clajps0n/ppo-CartPole-v1"

## Define the commit message
commit_message = "Upload PPO CartPole-v1 trained agent"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: Monitor(gymnasium.make(env_id, render_mode="rgb_array"))])

package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model
               model_architecture=model_architecture, # The model architecture we used
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id,
               commit_message=commit_message)